In [ ]:
# default_exp ads

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
#hide
!pip install -qU nbdev fastcore

     |████████████████████████████████| 61kB 3.0MB/s 


In [ ]:
#hide
from pathlib import Path
path = Path("/content/drive/My Drive/upwork/Martin followers/tomorrowswig_followers")
%cd "{path}"

/content/drive/My Drive/upwork/Martin followers/tomorrowswig_followers


In [ ]:
#hide
#not deps but we need them to use nbdev and run tests
from nbdev import * 
from nbdev.showdoc import *
from fastcore.test import *

## Facebook Auth

In [ ]:
#hide
!pip install -qU gspread facebook_business pycountry

     |████████████████████████████████| 1.0MB 4.1MB/s 
     |████████████████████████████████| 10.1MB 13.6MB/s 


In [ ]:
#hide
%%capture

import yaml
with open(path.parent/"facebook_creds.yml") as f:
    data = yaml.safe_load(f)
    %env APP_ID {data.get("APP_ID")}
    %env APP_SECRET {data.get("APP_SECRET")}
    %env TOKEN {data.get("TOKEN")}
    %env AD_ACC_ID {data.get("AD_ACC_ID")}

In [ ]:
#export
from tomorrowswig_followers.core import *

In [ ]:
#export
import os
from typing import *

from datetime import datetime, timedelta
from functools import partial

import pandas as pd
import numpy as np
from facebook_business.api import FacebookAdsApi
from facebook_business.adobjects.adaccount import AdAccount
import pycountry

AD_ACC_ID = os.environ.get("AD_ACC_ID")

In [ ]:
APP_ID = os.environ.get("APP_ID")
APP_SECRET = os.environ.get("APP_SECRET")
TOKEN = os.environ.get("TOKEN")
USER_ID = os.environ.get("USER_ID")

## Ads reporting

In [ ]:
#export
FacebookAdsApi.init(APP_ID, APP_SECRET, TOKEN)
account = AdAccount(AD_ACC_ID)

In [ ]:
datetime.fromtimestamp(1595946718)

datetime.datetime(2020, 7, 28, 14, 31, 58)

In [ ]:
datetime(2020,7,20,0,0).strftime('%s')

'1595203200'

In [ ]:
insights = account.get_insights(fields=[
    "ad_name",
    "clicks",
    "cpc",
    "ctr",
    "cpm",
    "cost_per_action_type",
    # AdsInsights.Field.cost_per_one_thousand_ad_impression, permissions
    "spend",
    # "adset_budget_value", permissions
    "impressions",
    "reach",
    "actions",
    "video_avg_time_watched_actions",
    "video_p50_watched_actions",
    "video_p75_watched_actions",
    "video_p95_watched_actions",

], params={
    'level': "ad",
    'date_preset': "yesterday",
    # "date_start": "1595116800",
    # "date_stop": "1595203200"
})


In [ ]:
def get_action(cell, name):
    action = [a for a in cell if a["action_type"] == name]
    if action:
        return action[0]["value"]
    return 0

In [ ]:
# fields = [f for f in AdsInsights.Field.__dict__ if not f.startswith("__")]

In [ ]:
# account.get_insights(fields=[fields[53]], params={
#     'level': AdsInsights.Level.ad,
#     'date_preset': AdsInsights.DatePreset.last_3d,
# })[-1]

In [ ]:
# account.get_insights(fields=fields[:53], params={
#     'level': AdsInsights.Level.ad,
#     'date_preset': AdsInsights.DatePreset.last_3d,
# })

In [ ]:
# account.get_insights(
#     # fields=fields[:53],
#     fields = ["actions"],
#     params={
#     'level': "ad",
#     'date_preset': "this_week_sun_today",
# })

In [ ]:
#export
def get_dif(df: pd.DataFrame) -> pd.DataFrame:
    change_df = df.diff(axis=1, periods=-1)
    change_df = change_df.fillna(0).astype(int)
    return change_df

In [ ]:
#export
# "Relevance Score",
COLUMNS = ["Ad Name", "New Followers", "% (Clicks)", "% (Impressions)", "Cost Per Follow",
           "Clicks (All)", "Link Clicks", "CPC (All)" , "CPC (Cost per Link Click)",
           "CTR (All)", "CPM (Cost per 1,000 Impressions)", "Amount Spent (USD)", "Impressions", "Reach", "Post Reactions", "Post Shares",
           "Video Average Play Time", "Video Plays at 50%", "Video Plays at 75%", "Video Plays at 95%"]

In [ ]:
# write_df(df, f"test {worksheet_name}")

In [ ]:
#export
get_insights = partial(account.get_insights, fields=[
        "ad_name",
        "clicks",
        "cpc",
        "ctr",
        "cpm",
        "cost_per_action_type",
        "spend",
        "impressions",
        "reach",
        "actions",
        "video_avg_time_watched_actions",
        "video_p50_watched_actions",
        "video_p75_watched_actions",
        "video_p95_watched_actions",
    ], params={
        'level': "ad",
        'date_preset': "yesterday",
    })

In [ ]:
#export
def get_action(cell, name):
    action = [a for a in cell if a["action_type"] == name]
    if action:
        return action[0]["value"]
    return 0

In [ ]:
#export
def get_followers_change(date: datetime) -> pd.DataFrame:
    history_df = get_df("History")
    new_followers = get_dif(history_df)
    followers_date = (date + timedelta(days=1)).strftime("%b %d")
    new_followers = new_followers.iloc[:,new_followers.columns.str.startswith(followers_date)].replace(0, np.nan)
    return new_followers.dropna(axis=1, how="all").iloc[:, :1].dropna().astype(int)

In [ ]:
assert get_followers_change(datetime.strptime("2020-07-25", "%Y-%m-%d")).columns[0] == "Jul 26                 2020-07-26 19:10:34"

In [ ]:
#export
def get_insights_df(insights: List) -> Tuple[pd.DataFrame, str]:
    df = pd.DataFrame(insights)
    floats = ["cpc", "cpm", "ctr", "spend"]
    df[floats] = df[floats].astype(float).round(2)
    ints = ["clicks", "impressions", "reach"]
    df[ints] = df[ints].astype(int)
    index = df["ad_name"].apply(lambda x: pycountry.countries.search_fuzzy(x.split("-")[1].strip())[0].alpha_2)
    index.name = "countries"
    df.set_index(index, inplace=True)
    df.sort_values("ad_name", inplace=True)

    date = datetime.strptime(df["date_start"].values[0], "%Y-%m-%d")
    worksheet_name = date.strftime("%b %d %Y")

    df["Post Reactions"] = df["actions"].apply(partial(get_action, name="post_reaction"))
    df["Post Shares"] = df["actions"].apply(partial(get_action, name="post"))
    df["Link Clicks"] = df["actions"].apply(partial(get_action, name="link_click"))
    df["CPC (Cost per Link Click)"] = df["cost_per_action_type"].apply(partial(get_action, name="link_click")).astype(float)
    get_video_action = partial(get_action, name="video_view")
    df["Video Average Play Time"] = df["video_avg_time_watched_actions"].apply(get_video_action)
    df["Video Plays at 50%"] = df["video_p50_watched_actions"].apply(get_video_action)
    df["Video Plays at 75%"] = df["video_p75_watched_actions"].apply(get_video_action)
    df["Video Plays at 95%"] = df["video_p95_watched_actions"].apply(get_video_action)

    df.drop(columns=["actions", "cost_per_action_type", "date_start", "date_stop",
                     "video_avg_time_watched_actions"] +
            [f"video_p{p}_watched_actions" for p in [50,75,95]], inplace=True)
    
    df["New Followers"] = get_followers_change(date)

    df["% (Clicks)"] = (df["New Followers"] / df["clicks"]* 100).round().astype(int)
    df["% (Impressions)"] = (df["New Followers"] / df["impressions"] * 100)
    df["Cost Per Follow"] = (df["spend"] / df["New Followers"])
    to_round = ["Cost Per Follow", "% (Impressions)", "CPC (Cost per Link Click)"]
    df[to_round] = df[to_round].round(2)
    df.rename(columns={"ad_name": "Ad Name", "clicks": "Clicks (All)", "cpc": "CPC (All)",
                   "ctr": "CTR (All)", "cpm": "CPM (Cost per 1,000 Impressions)",
                   "reach": "Reach", "impressions": "Impressions", "spend": "Amount Spent (USD)"}, inplace=True)
    return df[COLUMNS].replace(0, ""), worksheet_name

In [ ]:
#export

def create_insights(event: Dict = None, context=None,) -> str:
    insights = get_insights()
    df, worksheet_name = get_insights_df(insights)
    write_df(df, f"{worksheet_name}")
    return "Success"

In [ ]:
create_insights()

'Success'

In [ ]:
## TODO
## fix same country
## unfollowers
## relevance score
## year in history

In [ ]:
#hide
from nbdev.export import notebook2script
notebook2script()

Converted 00_core.ipynb.
Converted 01_followers.ipynb.
Converted 02_ads.ipynb.
Converted index.ipynb.
